In [1]:
from pathlib import Path
from utils import set_up_logging
import os
import polars as pl
import shutil
from tqdm import tqdm

set_up_logging(Path("../logs"))

SOURCE_PATH = Path("/mnt/wsl/PHYSICALDRIVE0p1/downloaded-unsplash")
TARGET_PATH = Path("/mnt/wsl/PHYSICALDRIVE0p1/featured")

SOURCE_PATH = SOURCE_PATH.resolve()
assert SOURCE_PATH.exists()
shutil.rmtree(TARGET_PATH, ignore_errors=True)
TARGET_PATH.mkdir(exist_ok=True)

photos = (
    pl.scan_csv(
        "../data/unsplash-full/photos.tsv000",
        separator="\t",
        infer_schema_length=100000,
    )
    .filter(pl.col("photo_featured") == "t")
    .sort("photo_id")
    .select("photo_id", "photo_image_url")
    .collect()
)

f"Selected {len(photos)} photos as mask"

'Selected 294403 photos as mask'

In [2]:
all_paths = list(SOURCE_PATH.glob("*"))
f"{len(all_paths)} downloaded photos found"

'1927632 downloaded photos found'

In [3]:
files = pl.DataFrame(
    {
        "photo_id": [path.name.split(".")[0] for path in all_paths],
        "path": [str(path) for path in all_paths],
    }
)
files.limit(10)

photo_id,path
str,str
"""HjVrVy3KTPM""","""/mnt/wsl/PHYSICALDRIVE0p1/down…"
"""0tQX63dH_oU""","""/mnt/wsl/PHYSICALDRIVE0p1/down…"
"""HFOjFLIgE64""","""/mnt/wsl/PHYSICALDRIVE0p1/down…"
"""1TiRQeEgzlM""","""/mnt/wsl/PHYSICALDRIVE0p1/down…"
"""85HrJ4N00dg""","""/mnt/wsl/PHYSICALDRIVE0p1/down…"
"""5ouomWc_8kY""","""/mnt/wsl/PHYSICALDRIVE0p1/down…"
"""1Zo2W5tW-VU""","""/mnt/wsl/PHYSICALDRIVE0p1/down…"
"""DG7Bv6V6-2Q""","""/mnt/wsl/PHYSICALDRIVE0p1/down…"
"""GLmohhX8vVQ""","""/mnt/wsl/PHYSICALDRIVE0p1/down…"


In [4]:
selected = files.join(photos, on="photo_id", how="inner")
f"{len(selected)} photos found in the mask locally downloaded"

'294367 photos found in the mask locally downloaded'

In [5]:
for path in tqdm(selected["path"]):
    os.symlink(path, TARGET_PATH / path.split("/")[-1])

100%|██████████| 294367/294367 [30:04<00:00, 163.09it/s]   
